Code to Generate Demo CSV

In [ ]:
# Corrected Snippet A: Demo CSV generator + download button (Colab-friendly)
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import io
from google.colab import files
import IPython.display as disp

def generate_demo_sales(n_rows: int = 800, seed: int = 42) -> pd.DataFrame:
    random.seed(seed)
    np.random.seed(seed)
    products = [
        ("P-100","Widget A","Hardware"),
        ("P-101","Widget B","Hardware"),
        ("P-200","Service X","Services"),
        ("P-201","Service Y","Services"),
        ("P-300","Accessory Q","Accessories"),
        ("P-301","Accessory R","Accessories"),
        ("P-400","Enterprise Suite","Software"),
        ("P-401","SMB Suite","Software"),
        ("P-500","Addon 1","Addons"),
    ]
    salespeople = ["Alice","Bob","Carla","Dustin","Eve","Farah","George"]
    regions = ["North America","Europe","Asia","South America","Oceania"]
    country_map = {
        "North America":["USA","CAN","MEX"],
        "Europe":["GBR","DEU","FRA","ESP","ITA"],
        "Asia":["IND","CHN","JPN","SGP","PAK"],
        "South America":["BRA","ARG","COL"],
        "Oceania":["AUS","NZL"]
    }
    cities = {
        "USA":["New York","San Francisco","Chicago"],
        "CAN":["Toronto","Vancouver"],
        "GBR":["London","Manchester"],
        "DEU":["Berlin","Munich"],
        "FRA":["Paris","Lyon"],
        "IND":["Bengaluru","Mumbai"],
        "CHN":["Shanghai","Beijing"],
        "JPN":["Tokyo","Osaka"],
        "SGP":["Singapore"],
        "PAK":["Karachi","Lahore"],
        "BRA":["Sao Paulo","Rio de Janeiro"],
        "ARG":["Buenos Aires"],
        "COL":["Bogota"],
        "AUS":["Sydney","Melbourne"],
        "NZL":["Auckland"],
        "MEX":["Mexico City"],
        "ESP":["Madrid"],
        "ITA":["Rome"],
    }
    channels = ["Website","Partner","Email","Event","Paid Ads"]
    stages = ["Lead","Qualified","Proposal","Negotiation","Closed Won","Closed Lost"]

    # customers
    n_customers = max(200, n_rows // 4)
    customers = [f"CUST{idx:05d}" for idx in range(1, n_customers+1)]
    today = pd.Timestamp.today().normalize()
    # create first_purchase_dates as pandas Timestamps
    first_purchase_choices = pd.date_range(today - pd.DateOffset(months=30), today)
    first_purchase_dates = pd.to_datetime(
        np.random.choice(first_purchase_choices, size=n_customers)
    )
    cust_df = pd.DataFrame({
        "customer_id": customers,
        "first_purchase_date": first_purchase_dates
    })

    rows = []
    order_id = 1
    opp_counter = 1
    for _ in range(n_rows):
        cust = np.random.choice(customers)
        # ensure we get a pandas Timestamp for first purchase date
        first_pd_raw = cust_df.loc[cust_df['customer_id'] == cust, 'first_purchase_date'].values[0]
        first_pd = pd.to_datetime(first_pd_raw)  # <-- conversion fixes numpy.datetime64 vs Timestamp

        lead_date = pd.Timestamp(today - pd.Timedelta(days=int(np.random.exponential(scale=120))))
        close_lag = int(np.clip(np.random.exponential(scale=14), 0, 120))
        close_date = lead_date + pd.Timedelta(days=close_lag)
        if close_date > today:
            close_date = today - pd.Timedelta(days=random.randint(0,5))
        order_date = close_date

        p = random.choice(products)
        product_id, product_name, category = p
        region = random.choice(regions)
        country = random.choice(country_map[region])
        city = random.choice(cities.get(country, ["Unknown"]))
        salesperson = random.choice(salespeople)
        units = np.random.poisson(3) + 1
        base_price_map = {"Hardware":400,"Services":1500,"Accessories":50,"Software":5000,"Addons":200}
        base_price = base_price_map.get(category, 300)
        price = max(20, float(np.random.normal(base_price, base_price*0.4)))
        revenue = round(units * price * (1 - np.random.choice([0.0,0.05,0.1,0.2], p=[0.7,0.15,0.1,0.05])), 2)
        stage = np.random.choice(stages, p=[0.25,0.2,0.18,0.12,0.18,0.07])
        channel = np.random.choice(channels, p=[0.4,0.2,0.2,0.1,0.1])
        # opportunity id: some grouping
        if random.random() < 0.15:
            opp = f"OPP{opp_counter:06d}"
            opp_counter += 1
        else:
            opp = f"OPP{random.randint(1, max(1, opp_counter)) :06d}"

        rows.append({
            "order_id": f"ORD{order_id:07d}",
            "opportunity_id": opp,
            "customer_id": cust,
            "order_date": order_date.date().isoformat(),
            "lead_date": lead_date.date().isoformat(),
            "close_date": close_date.date().isoformat(),
            "first_purchase_date": first_pd.date().isoformat(),  # safe: first_pd is a pd.Timestamp
            "revenue": revenue,
            "units": units,
            "product_id": product_id,
            "product_name": product_name,
            "category": category,
            "salesperson": salesperson,
            "region": region,
            "country": country,
            "city": city,
            "stage": stage,
            "channel": channel
        })
        order_id += 1

    df = pd.DataFrame(rows)
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['first_purchase_date'] = pd.to_datetime(df['first_purchase_date'])
    df['is_returning'] = (df['order_date'] > df['first_purchase_date']).astype(int)
    df['aov'] = df['revenue']  # order-level
    df['sales_cycle_days'] = (pd.to_datetime(df['close_date']) - pd.to_datetime(df['lead_date'])).dt.days.clip(lower=0)
    return df

# --- Usage: generate and download ---
demo_df = generate_demo_sales(n_rows=800)
print("Demo dataset generated. Rows:", len(demo_df))
display(demo_df.head(5))

# Save to CSV in-memory and provide download via google.colab.files
csv_buffer = io.StringIO()
demo_df.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)
with open("continuumai_demo_sales.csv", "w", encoding="utf-8") as f:
    f.write(csv_buffer.getvalue())
print("Saved demo CSV to workspace: continuumai_demo_sales.csv")
# Trigger browser download in Colab
files.download("continuumai_demo_sales.csv")
